# Evaluation of UCDD on AGRAW1

## Imports

In [1]:
import numpy as np
import pandas as pd
import sklearn
import sys

## AGRAW1 dataset locations

In [2]:
abrupt_agraw1_path = '../Datasets_concept_drift/synthetic_data/abrupt_drift/agraw1_1_abrupt_drift_0_noise_balanced.arff'
gradual_agraw1_paths = [
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_05.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_1.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_5.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_10.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_20.arff'
]

all_agraw1_data_paths = [abrupt_agraw1_path] + gradual_agraw1_paths

## Accept and preprocess AGRAW1 datasets

In [3]:
from eval_helpers import accepting
from sklearn.preprocessing import MinMaxScaler
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split


agraw1_exclude_reference_batches = {}
agraw1_exclude_testing_batches = {}
agraw1_onehot_reference_batches = {}
agraw1_onehot_testing_batches = {}
agraw1_target_reference_batches = {}
agraw1_target_testing_batches = {}


# agraw1 with categories excluded
for file_path in all_agraw1_data_paths:
    df_x, df_y = accepting.get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = accepting.divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = accepting.divide_numeric_categorical(df_x_test)
    
    reference_data = df_x_ref_num.to_numpy()
    testing_data = df_x_test_num.to_numpy()
    scaler = MinMaxScaler()
    scaler.fit(reference_data)
    reference_data = scaler.transform(reference_data)
    testing_data = scaler.transform(testing_data)
    
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw1_exclude_reference_batches[file_path] = ref_batches
    agraw1_exclude_testing_batches[file_path] = test_batches
    
print('agraw1 exclude')
print(agraw1_exclude_reference_batches)
print(agraw1_exclude_testing_batches)

# agraw1 with categories onehot encoded
for file_path in all_agraw1_data_paths:
    df_x, df_y = accepting.get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = accepting.divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = accepting.divide_numeric_categorical(df_x_test)
    
    ref_index = df_x_ref_cat.index
    test_index = df_x_test_cat.index
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(df_x_ref_cat)
    df_x_ref_cat_transformed = pd.DataFrame(encoder.transform(df_x_ref_cat))
    df_x_test_cat_transformed = pd.DataFrame(encoder.transform(df_x_test_cat))
    df_x_ref_cat_transformed.set_index(ref_index, inplace=True)
    df_x_test_cat_transformed.set_index(test_index, inplace=True)
    
    reference_data = df_x_ref_num.join(df_x_ref_cat_transformed, lsuffix='_num').to_numpy()
    testing_data = df_x_test_num.join(df_x_test_cat_transformed, lsuffix='_num').to_numpy()
    scaler = MinMaxScaler()
    scaler.fit(reference_data)
    reference_data = scaler.transform(reference_data)
    testing_data = scaler.transform(testing_data)
    
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw1_onehot_reference_batches[file_path] = ref_batches
    agraw1_onehot_testing_batches[file_path] = test_batches
    
print('agraw1 onehot')
print(agraw1_onehot_reference_batches)
print(agraw1_onehot_testing_batches)

# agraw1 with categories target encoded
for file_path in all_agraw1_data_paths:
    df_x, df_y = accepting.get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = accepting.divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = accepting.divide_numeric_categorical(df_x_test)
    
    ref_index = df_x_ref_cat.index
    test_index = df_x_test_cat.index
    encoder = TargetEncoder()
    encoder.fit(df_x_ref_cat, df_y_ref)
    df_x_ref_cat_transformed = pd.DataFrame(encoder.transform(df_x_ref_cat))
    df_x_test_cat_transformed = pd.DataFrame(encoder.transform(df_x_test_cat))
    df_x_ref_cat_transformed.set_index(ref_index, inplace=True)
    df_x_test_cat_transformed.set_index(test_index, inplace=True)
    
    reference_data = df_x_ref_num.join(df_x_ref_cat_transformed, lsuffix='_num').to_numpy()
    testing_data = df_x_test_num.join(df_x_test_cat_transformed, lsuffix='_num').to_numpy()
    scaler = MinMaxScaler()
    scaler.fit(reference_data)
    reference_data = scaler.transform(reference_data)
    testing_data = scaler.transform(testing_data)
    
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw1_target_reference_batches[file_path] = ref_batches
    agraw1_target_testing_batches[file_path] = test_batches
    
print('agraw1 target')
print(agraw1_target_reference_batches)
print(agraw1_target_testing_batches)

df               salary    commission   age     elevel       car      zipcode  \
0      115014.164791      0.000000  56.0  b'level3'  b'car15'  b'zipcode5'   
1       39855.706174  20372.573526  73.0  b'level2'   b'car3'  b'zipcode7'   
2       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
3       57795.185560  13344.143894  80.0  b'level2'   b'car1'  b'zipcode3'   
4       46602.621787  26569.114334  46.0  b'level0'  b'car13'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995  142923.630833      0.000000  28.0  b'level0'  b'car13'  b'zipcode3'   
99996   48591.770745  22774.455785  24.0  b'level2'   b'car7'  b'zipcode1'   
99997   40282.441180  70705.062385  24.0  b'level0'  b'car14'  b'zipcode9'   
99998   24701.771322  17773.082776  70.0  b'level2'  b'car14'  b'zipcode1'   
99999  101356.827789      0.000000  63.0  b'level3'  b'car20'  b'zipcode3'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\ucdd_improved\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df               salary    commission   age     elevel       car      zipcode  \
0      115014.164791      0.000000  56.0  b'level3'  b'car15'  b'zipcode5'   
1       39855.706174  20372.573526  73.0  b'level2'   b'car3'  b'zipcode7'   
2       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
3       57795.185560  13344.143894  80.0  b'level2'   b'car1'  b'zipcode3'   
4       46602.621787  26569.114334  46.0  b'level0'  b'car13'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   20450.346419  50131.775485  68.0  b'level2'  b'car18'  b'zipcode7'   
99996  142923.630833      0.000000  28.0  b'level0'  b'car13'  b'zipcode3'   
99997   48591.770745  22774.455785  24.0  b'level2'   b'car7'  b'zipcode1'   
99998   40282.441180  70705.062385  24.0  b'level0'  b'car14'  b'zipcode9'   
99999   24701.771322  17773.082776  70.0  b'level2'  b'car14'  b'zipcode1'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\ucdd_improved\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df               salary    commission   age     elevel       car      zipcode  \
0      115014.164791      0.000000  56.0  b'level3'  b'car15'  b'zipcode5'   
1       39855.706174  20372.573526  73.0  b'level2'   b'car3'  b'zipcode7'   
2       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
3       57795.185560  13344.143894  80.0  b'level2'   b'car1'  b'zipcode3'   
4       46602.621787  26569.114334  46.0  b'level0'  b'car13'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   82084.560570      0.000000  38.0  b'level1'   b'car8'  b'zipcode1'   
99996  122185.589049      0.000000  68.0  b'level2'  b'car16'  b'zipcode5'   
99997   75112.189101      0.000000  43.0  b'level0'  b'car14'  b'zipcode3'   
99998   77645.133039      0.000000  78.0  b'level4'   b'car9'  b'zipcode6'   
99999   20450.346419  50131.775485  68.0  b'level2'  b'car18'  b'zipcode7'   

             hvalue  hyears           loan      class  
0   

...

df               salary    commission   age     elevel       car      zipcode  \
0      115014.164791      0.000000  56.0  b'level3'  b'car15'  b'zipcode5'   
1       39855.706174  20372.573526  73.0  b'level2'   b'car3'  b'zipcode7'   
2       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
3       57795.185560  13344.143894  80.0  b'level2'   b'car1'  b'zipcode3'   
4       46602.621787  26569.114334  46.0  b'level0'  b'car13'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   37724.568475  11570.033669  27.0  b'level1'   b'car8'  b'zipcode3'   
99996  110548.635896      0.000000  58.0  b'level4'  b'car20'  b'zipcode2'   
99997  119805.259163      0.000000  73.0  b'level1'  b'car15'  b'zipcode1'   
99998  120012.609799      0.000000  48.0  b'level1'   b'car9'  b'zipcode7'   
99999  120413.864305      0.000000  38.0  b'level4'   b'car9'  b'zipcode3'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\ucdd_improved\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encod

df               salary    commission   age     elevel       car      zipcode  \
0      115014.164791      0.000000  56.0  b'level3'  b'car15'  b'zipcode5'   
1       39855.706174  20372.573526  73.0  b'level2'   b'car3'  b'zipcode7'   
2       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
3       57795.185560  13344.143894  80.0  b'level2'   b'car1'  b'zipcode3'   
4       46602.621787  26569.114334  46.0  b'level0'  b'car13'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   86287.261924      0.000000  65.0  b'level2'  b'car18'  b'zipcode4'   
99996   78633.802398      0.000000  48.0  b'level2'  b'car11'  b'zipcode4'   
99997   99038.707844      0.000000  79.0  b'level1'  b'car16'  b'zipcode8'   
99998  120472.854608      0.000000  40.0  b'level3'  b'car16'  b'zipcode6'   
99999  128351.871620      0.000000  56.0  b'level2'   b'car6'  b'zipcode7'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\ucdd_improved\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encod

agraw1 target
{'../Datasets_concept_drift/synthetic_data/abrupt_drift/agraw1_1_abrupt_drift_0_noise_balanced.arff': [array([[0.73089198, 0.        , 0.6       , ..., 0.27295809, 0.85304062,
        0.63548506],
       [0.15273383, 0.27168449, 0.88333333, ..., 0.14991113, 0.3861165 ,
        0.20596598],
       [0.52078707, 0.        , 0.53333333, ..., 0.        , 0.01822838,
        0.63548506],
       ...,
       [0.79527985, 0.        , 0.7       , ..., 0.        , 1.        ,
        0.20596598],
       [0.62023278, 0.        , 0.56666667, ..., 0.27295809, 0.23885852,
        0.05969598],
       [0.72150735, 0.        , 0.83333333, ..., 0.50887558, 0.76350635,
        0.63548506]]), array([[0.19878044, 0.96414871, 0.55      , ..., 0.14991113, 0.85304062,
        0.20596598],
       [0.86428378, 0.        , 0.05      , ..., 0.50887558, 0.82625127,
        0.        ],
       [0.08956815, 0.36984179, 0.45      , ..., 0.50887558, 0.76350635,
        0.50822506],
       ...,
       [0.1

## Helper function

In [4]:
from eval_helpers import kmeans_verbose_helpers


def write_kmeans_results_ucdd_helper(output_filename_no_extension, ref_batches, n_init, max_iter, tol, random_state):
    # dummy = [np.asarray(1), np.asarray(2), np.asarray(3)]
    combinations = []
    for i in range(3):
    #     combinations.append(np.vstack((dummy[i], dummy[(i + 1) % 3])))
        combinations.append(np.vstack((ref_batches[i], ref_batches[(i + 1) % 3])))
        
    all_results_from_combinations = []
    for i, combination in enumerate(combinations):
        filename = output_filename_no_extension + str(i) + '.txt'
        print('filename', filename)
        kmeans_verbose_helpers.write_verbose_kmeans_to_file(result_filename=output_filename_no_extension + str(i) + '.txt',
                                     data_to_cluster=combination,
                                     n_clusters=2, n_init=n_init, max_iter=max_iter, tol=tol, random_state=random_state)
        output_dicts = kmeans_verbose_helpers.convert_kmeans_output_file_to_dicts(filename, n_init=n_init)
        all_results_from_combinations.append(output_dicts)
        kmeans_verbose_helpers.print_stats_from_kmeans_output_dicts(output_dicts)
        
    kmeans_verbose_helpers.print_stats_from_all_combinations(all_results_from_combinations)

something


## AGRAW1 with categories excluded

### Find the best tol and max_iter (the drift type is irrelevant)

In [7]:
write_kmeans_results_ucdd_helper('agraw1_exclude_new_output', agraw1_exclude_reference_batches[abrupt_agraw1_path],
                                 n_init=100, max_iter=500, tol=0,
                                 random_state=1053)

filename agraw1_exclude_new_output0.txt
random state: 1053
total number of results: 100
maximum number of iterations: 18
minimum initial inertia: 8347.093515830069
maximum initial inertia: 18690.074352335017
number of unique final inertia values: 3
minimum final inertia: 6832.419247273033
maximum final inertia: 6832.419270386381
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0
filename agraw1_exclude_new_output1.txt
random state: 1053
total number of results: 100
maximum number of iterations: 18
minimum initial inertia: 8747.52244428163
maximum initial inertia: 17134.178458342543
number of unique final inertia values: 3
minimum final inertia: 6820.074930844902
maximum final inertia: 6820.0750120817775
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0
filename agraw1_exclude_new_output2.txt
random state: 1053
total number of results: 100
maximum number of iterations: 15
minimum

### Use them for the analysis

In [5]:
from core import ucdd_eval


agraw1_exclude_stats = {}
agraw1_path = all_agraw1_data_paths[-1]
# for agraw1_path in all_agraw1_data_paths:
#     runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
#         ucdd_eval.all_drifting_batches_randomness_robust(
#         agraw1_reference_batches[agraw1_path],
#         agraw1_testing_batches[agraw1_path],
#         min_ref_batches_drift=0.3,
#         additional_check=True,
#         n_init=100,
#         max_iter=18000,
#         tol=0,
#         true_drift_idx=2,
#         min_runs=2
#     )
#     agraw1_stats[agraw1_path] = {
#         'runs_results_bool': runs_results_bool,
#         'final_fpr_mean': final_fpr_mean,
#         'fpr_std_err': fpr_std_err,
#         'final_latency_mean': final_latency_mean,
#         'latency_std_err': latency_std_err
#     }
    
for agraw1_path in all_agraw1_data_paths:
    runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
        ucdd_eval.all_drifting_batches_randomness_robust(
        agraw1_exclude_reference_batches[agraw1_path],
        agraw1_exclude_testing_batches[agraw1_path],
        min_ref_batches_drift=0.6,
        additional_check=True,
        n_init=100,
        max_iter=18000,
        tol=0,
        true_drift_idx=2,
        min_runs=2
    )
    agraw1_exclude_stats[agraw1_path] = {
        'runs_results_bool': runs_results_bool,
        'final_fpr_mean': final_fpr_mean,
        'fpr_std_err': fpr_std_err,
        'final_latency_mean': final_latency_mean,
        'latency_std_err': latency_std_err
    }

print('AGRAW1 STATS')
print(agraw1_exclude_stats)

random_state
0
#### TEST BATCH 0 of 7 ####
#### TEST BATCH 1 of 7 ####
#### TEST BATCH 2 of 7 ####
#### TEST BATCH 3 of 7 ####
#### TEST BATCH 4 of 7 ####
#### TEST BATCH 5 of 7 ####
#### TEST BATCH 6 of 7 ####
random_state
100
#### TEST BATCH 0 of 7 ####
#### TEST BATCH 1 of 7 ####
#### TEST BATCH 2 of 7 ####
#### TEST BATCH 3 of 7 ####
#### TEST BATCH 4 of 7 ####
#### TEST BATCH 5 of 7 ####
#### TEST BATCH 6 of 7 ####
fpr s.e. 0.0
latency s.e. 0.0
final fpr mean 0.0
final latency mean 1.0
random_state
0
#### TEST BATCH 0 of 7 ####
#### TEST BATCH 1 of 7 ####


KeyboardInterrupt: 

### Save the obtained results to csv

In [ ]:
from eval_helpers import helpers


final_result_dict = {
    'type_of_data': [], 'dataset': [], 'drift': [], 'width': [], 'encoding': [],
    'min_ref_batches_drift': [], 'additional_check': [],
    'n_init': [], 'max_iter': [], 'tol': [],
    'FPR_mean': [], 'latency_mean': []
}

for data_path, stats_dict in agraw1_exclude_stats.items():
    synthetic_filename_info = helpers.synthetic_data_information(data_path)
    encoding = 'exclude'
    fpr_mean = float(stats_dict['final_fpr_mean'])
    latency_mean = float(stats_dict['final_latency_mean'])
    
    final_result_dict['type_of_data'].append(synthetic_filename_info['type_of_data'])
    final_result_dict['dataset'].append(synthetic_filename_info['dataset_name'])
    final_result_dict['drift'].append(synthetic_filename_info['drift_type'])
    final_result_dict['width'].append(synthetic_filename_info['drift_width'])
    final_result_dict['encoding'].append(encoding)
    final_result_dict['min_ref_batches_drift'].append(0.3)
    final_result_dict['additional_check'].append('yes')
    final_result_dict['n_init'].append(100)
    final_result_dict['max_iter'].append(18000)
    final_result_dict['tol'].append(0)
    final_result_dict['FPR_mean'].append(fpr_mean)
    final_result_dict['latency_mean'].append(latency_mean)
    
final_result_df = pd.DataFrame.from_dict(final_result_dict)
sorted_final_result_df = final_result_df.sort_values(['drift', 'dataset', 'encoding', 'width'])
final_result_df.to_csv('agraw1_exclude_jupyter_results.csv', index=False)

## AGRAW1 with categories onehot encoded

### Find the best tol and max_iter (the drift type is irrelevant)

In [9]:
write_kmeans_results_ucdd_helper('agraw1_onehot_new_output', agraw1_onehot_reference_batches[abrupt_agraw1_path],
                                 n_init=100, max_iter=500, tol=0,
                                 random_state=1053)

filename agraw1_onehot_new_output0.txt
random state: 1053
total number of results: 100
maximum number of iterations: 34
minimum initial inertia: 104954.65523014727
maximum initial inertia: 116645.01047187511
number of unique final inertia values: 19
minimum final inertia: 58341.71044879218
maximum final inertia: 59761.482370558224
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0
filename agraw1_onehot_new_output1.txt
random state: 1053


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


total number of results: 100
maximum number of iterations: 33
minimum initial inertia: 105187.473271114
maximum initial inertia: 117235.7046827969
number of unique final inertia values: 13
minimum final inertia: 58307.80854907961
maximum final inertia: 59757.571876885886
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0
filename agraw1_onehot_new_output2.txt
random state: 1053
total number of results: 100
maximum number of iterations: 26
minimum initial inertia: 104976.092943036
maximum initial inertia: 118204.3234742695
number of unique final inertia values: 23
minimum final inertia: 58330.68910892084
maximum final inertia: 59758.880442981055
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0
{'total_max_iterations': 34, 'total_min_init_inertia': 104954.65523014727, 'total_max_init_inertia': 118204.3234742695, 'total_min_final_inertia': 58307.80854907961, 'total_max_final_inert

### Use them for the analysis

In [10]:
from core import ucdd_eval


print('...')
agraw1_onehot_stats = {}

for agraw1_path in all_agraw1_data_paths:
    runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
        ucdd_eval.all_drifting_batches_randomness_robust(
        agraw1_onehot_reference_batches[agraw1_path],
        agraw1_onehot_testing_batches[agraw1_path],
        min_ref_batches_drift=0.6,
        additional_check=True,
        n_init=100,
        max_iter=34000,
        tol=0,
        true_drift_idx=2,
        min_runs=2
    )
    agraw1_onehot_stats[agraw1_path] = {
        'runs_results_bool': runs_results_bool,
        'final_fpr_mean': final_fpr_mean,
        'fpr_std_err': fpr_std_err,
        'final_latency_mean': final_latency_mean,
        'latency_std_err': latency_std_err
    }

print('AGRAW1 STATS')
print(agraw1_onehot_stats)

...
train_batch_strategy TrainBatchStrategies.MAJORITY
random_state
0
n_init 100 max_iter 34000 tol 0
n_init 100 max_iter 34000 tol 0
n_init 100 max_iter 34000 tol 0


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


TrainBatchStrategies.MAJORITY
train_batch_strategy None
acceptable_strategies:
TrainBatchStrategies.ANY
TrainBatchStrategies.SUBMAJORITY
TrainBatchStrategies.MAJORITY
TrainBatchStrategies.ALL
n_init 100 max_iter 34000 tol 0
n_init 100 max_iter 34000 tol 0
n_init 100 max_iter 34000 tol 0
TrainBatchStrategies.MAJORITY
train_batch_strategy None
acceptable_strategies:
TrainBatchStrategies.ANY
TrainBatchStrategies.SUBMAJORITY
TrainBatchStrategies.MAJORITY
TrainBatchStrategies.ALL
n_init 100 max_iter 34000 tol 0
n_init 100 max_iter 34000 tol 0


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


n_init 100 max_iter 34000 tol 0


KeyboardInterrupt: 

### Save the obtained results to csv

In [23]:
from eval_helpers import helpers


final_result_dict = {
    'type_of_data': [], 'dataset': [], 'drift': [], 'width': [], 'encoding': [],
    'min_ref_batches_drift': [], 'additional_check': [],
    'n_init': [], 'max_iter': [], 'tol': [],
    'FPR_mean': [], 'latency_mean': []
}

for data_path, stats_dict in agraw1_onehot_stats.items():
    synthetic_filename_info = helpers.synthetic_data_information(data_path)
    encoding = 'onehot'
    fpr_mean = float(stats_dict['final_fpr_mean'])
    latency_mean = float(stats_dict['final_latency_mean'])
    
    final_result_dict['type_of_data'].append(synthetic_filename_info['type_of_data'])
    final_result_dict['dataset'].append(synthetic_filename_info['dataset_name'])
    final_result_dict['drift'].append(synthetic_filename_info['drift_type'])
    final_result_dict['width'].append(synthetic_filename_info['drift_width'])
    final_result_dict['encoding'].append(encoding)
    final_result_dict['min_ref_batches_drift'].append(0.6)
    final_result_dict['additional_check'].append('yes')
    final_result_dict['n_init'].append(100)
    final_result_dict['max_iter'].append(34000)
    final_result_dict['tol'].append(0)
    final_result_dict['FPR_mean'].append(fpr_mean)
    final_result_dict['latency_mean'].append(latency_mean)
    
final_result_df = pd.DataFrame.from_dict(final_result_dict)
sorted_final_result_df = final_result_df.sort_values(['drift', 'dataset', 'encoding', 'width'])
final_result_df.to_csv('agraw1_onehot_jupyter_results_majority.csv', index=False)

## AGRAW1 with categories target encoded

### Find the best tol and max_iter (the drift type is irrelevant)

In [11]:
write_kmeans_results_ucdd_helper('agraw1_target_new_output', agraw1_target_reference_batches[abrupt_agraw1_path],
                                 n_init=100, max_iter=500, tol=0,
                                 random_state=1053)

filename agraw1_target_new_output0.txt
random state: 1053
total number of results: 100
maximum number of iterations: 18
minimum initial inertia: 17607.98488650251
maximum initial inertia: 29056.647428908935
number of unique final inertia values: 21
minimum final inertia: 13299.613663873319
maximum final inertia: 14276.6351586479
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0
filename agraw1_target_new_output1.txt
random state: 1053
total number of results: 100
maximum number of iterations: 22
minimum initial inertia: 17728.884037917425
maximum initial inertia: 30906.112923249697
number of unique final inertia values: 13
minimum final inertia: 13290.390633663217
maximum final inertia: 14285.170980526265
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0
filename agraw1_target_new_output2.txt
random state: 1053


KeyboardInterrupt: 

### Use them for the analysis

In [ ]:
from core import ucdd_eval


agraw1_target_stats = {}
    
for agraw1_path in all_agraw1_data_paths:
    runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
        ucdd_eval.all_drifting_batches_randomness_robust(
        agraw1_target_reference_batches[agraw1_path],
        agraw1_target_testing_batches[agraw1_path],
        min_ref_batches_drift=0.6,
        additional_check=True,
        n_init=100,
        max_iter=45000,
        tol=0,
        true_drift_idx=2,
        min_runs=2
    )
    agraw1_target_stats[agraw1_path] = {
        'runs_results_bool': runs_results_bool,
        'final_fpr_mean': final_fpr_mean,
        'fpr_std_err': fpr_std_err,
        'final_latency_mean': final_latency_mean,
        'latency_std_err': latency_std_err
    }

print('AGRAW1 STATS')
print(agraw1_target_stats)

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


### Save the obtained results to csv

In [37]:
from eval_helpers import helpers


final_result_dict = {
    'type_of_data': [], 'dataset': [], 'drift': [], 'width': [], 'encoding': [],
    'min_ref_batches_drift': [], 'additional_check': [],
    'n_init': [], 'max_iter': [], 'tol': [],
    'FPR_mean': [], 'latency_mean': []
}

for data_path, stats_dict in agraw1_target_stats.items():
    synthetic_filename_info = helpers.synthetic_data_information(data_path)
    encoding = 'target'
    fpr_mean = float(stats_dict['final_fpr_mean'])
    latency_mean = float(stats_dict['final_latency_mean'])
    
    final_result_dict['type_of_data'].append(synthetic_filename_info['type_of_data'])
    final_result_dict['dataset'].append(synthetic_filename_info['dataset_name'])
    final_result_dict['drift'].append(synthetic_filename_info['drift_type'])
    final_result_dict['width'].append(synthetic_filename_info['drift_width'])
    final_result_dict['encoding'].append(encoding)
    final_result_dict['min_ref_batches_drift'].append(0.6)
    final_result_dict['additional_check'].append('yes')
    final_result_dict['n_init'].append(100)
    final_result_dict['max_iter'].append(45000)
    final_result_dict['tol'].append(0)
    final_result_dict['FPR_mean'].append(fpr_mean)
    final_result_dict['latency_mean'].append(latency_mean)
    
final_result_df = pd.DataFrame.from_dict(final_result_dict)
sorted_final_result_df = final_result_df.sort_values(['drift', 'dataset', 'encoding', 'width'])
final_result_df.to_csv('agraw1_target_jupyter_results_majority.csv', index=False)